# Zero to Hero

## 1.11 Arbol training/testing

El objetivo es calcular la ganancia en testing de un arbol de decision

In [5]:
options(warn=-1)

In [6]:
rm(list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1945483,104.0,3661321,195.6,3661321,195.6
Vcells,3253703,24.9,33548208,256.0,34834305,265.8


In [7]:
options(repr.plot.width=20, repr.plot.height=10) 
setwd("C:\\Users\\oliva\\OneDrive\\Desktop\\DS\\Austral\\08 - Labo1") #Aqui se debe poner la ruta de la PC local

In [8]:
dataset <- fread("./labo/datasets/paquete_premium_202011.csv")   #cargo el dataset

In [22]:
#creo training con 70%  y testing con 30%
set.seed(13)
train_rows <- createDataPartition(dataset$clase_ternaria, p = 0.70, list = FALSE)
dtrain  <- dataset[ train_rows]
dtest  <-  dataset[ -train_rows]
dtrain[, .N, clase_ternaria]
dtest[, .N, clase_ternaria]

clase_ternaria,N
CONTINUA,110761
BAJA+1,358
BAJA+2,454


clase_ternaria,N
CONTINUA,47468
BAJA+1,153
BAJA+2,194


##### ¿Por qué 70/30?
https://scholarworks.utep.edu/cgi/viewcontent.cgi?article=2202&context=cs_techrep

### Entrenamiento

In [10]:
param  <- list("cp"= -0.5,
               "minsplit"=  900,
               "minbucket"= 440,
               "maxdepth"= 5
              )

Entreno el modelo en los datos de training

In [13]:
#genero el modelo
modelo <-  rpart(formula = "clase_ternaria ~ .",
                        data = dtrain,
                        xval = 0, #no cross validation
                        control = param)

Aplico el modelo a los datos de testing

In [16]:
prediccion <- predict(modelo, dtest, type = "prob")
head(prediccion)

BAJA+1,BAJA+2,CONTINUA
3.693785e-04,1.015791e-03,0.9986148
5.291705e-05,2.645853e-05,0.9999206
1.488982e-04,7.444908e-04,0.9991066
2.256264e-03,3.443772e-03,0.9943000
3.693785e-04,1.015791e-03,0.9986148
1.040989e-02,1.886792e-02,0.9707222


### Calculo la ganancia del modelo en los datos de testing

A partir de la prediccion, calculo la probabilidad de BAJA+2 de cada registro de testing  **dtest**

In [15]:
prob_baja2  <- prediccion[, "BAJA+2"]
head(prob_baja2)

1            2            3            4            5            6 
1.015791e-03 2.645853e-05 7.444908e-04 3.443772e-03 1.015791e-03 1.886792e-02

Para el cálculo de la ganancia del modelo aplicado a testing debo tener en cuenta lo siguiente
* Solo envio estimulo a los registros que el modelo asigno una probabilidad mayor a 1/60
* Si no envio estimulo, no gano ni pierdo, es decir la ganacia es cero
* Si el registro al que envio el estímulo es un **BAJA+2**  entonces la ganancia es de +59000
* Si el registro al que envio el estímulo es un **BAJA+1**  entonces la ganancia es de  -1000
* Si el registro al que envio el estímulo es un **CONTINUA** entonces la ganancia es de  -1000

A la condición que solo envio a los registros con probabilidad mayor a 1/60 la resuelvo con la condicion lógica  (prob_baja2> 1/60)
<br>
Esta condicion termina siendo un 0 o un 1
<br>
vale 1 si y solo si se cumple la condicion, es decir  si prob_baja2 es mayor a 1/60

La ganancia de +59000 o -1000 la resuelvo de esta forma ifelse(clase_ternaria=="BAJA+2", 59000, -1000) -Esto se lee: ¿es clase ternaria igual a baja+2? Sí, entonces asignar 59000. No, entonces asignar -1000

Finalmente,  que se cumplan ambas condiciones al mismo tiempo, en este caso, es multiplicarlas, o sea
<br>
(prob_baja2>1/60) * ifelse( clase_ternaria=="BAJA+2", 59000, -1000)
<br>
La condicion anterior toma los siguientes valores

| prob(BAJA+2) | clase_ternaria | ganancia |
| --- | :---: | ---: |
| menor o igual a 1/60 | BAJA+1 | 0 |
| menor o igual a 1/60 | CONTINUA | 0 |
| menor o igual a 1/60 | BAJA+2 | 0 |
| mayor a 1/60 | BAJA+1 | -1000 |
| mayor a 1/60 | CONTINUA | -1000 |
| mayor a 1/60 | BAJA+2 | 59000 |


Finalmente hago el calculo en data.table

In [23]:
ganancia_testing <- dtest[ , sum((prob_baja2>1/60) * ifelse(clase_ternaria == "BAJA+2", 59000, -1000))]

In [24]:
ganancia_testing

[1] 3648000

Es importante que la ganancia esta calculada sobre los datos de testing, que en este caso son apenas el 30%
<br>
Si quiero extrapolar a todo el dataset, debo hacer el **cociente** de esa ganancia por 0.30   ( coloquialmente, debo dividir por 0.30 )

In [25]:
ganancia_testing_normalizada  <-  ganancia_testing/0.3

In [26]:
ganancia_testing_normalizada

[1] 12160000